In [ ]:
# !pip install pytorch-lightning

In [17]:
import numpy as np
import pandas as pd
from pathlib import Path

import os
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from collections import Counter
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

import pytorch_lightning as pl

from tqdm import tqdm_notebook as tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
train = pd.read_csv('../dataset/경진대회/train_data_dataframe.csv')
test = pd.read_csv('../dataset/경진대회/test_data_dataframe.csv')

In [26]:
train['text'] =  train['context'] + train['title'] + train['question']
test['text'] = test['context'] + test['title']  + test['question']

In [21]:
train.head(2)

,content_id,title,paragraph_id,context,question_id,question,answer_text,answer_start,is_impossible
0,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_RrlDreD2hE,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...",QUES_CHGHl0CHHK,경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활...,기술경쟁력,603.0,False
1,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_s5sSaKdCH3,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,QUES_ncJEA1ANCe,IMD평가에 포함되었던 기술관리부분의 한국의 순위는 평균 몇 위야,평균 32위,420.0,False


In [25]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [27]:
train['uncased_text'] = train['text'].apply(lambda x: x.lower())
test['uncased_text'] = test['text'].apply(lambda x: x.lower())


In [29]:
# Tokenize
train['tokenized_text'] = train['uncased_text'].apply(tokenizer.tokenize)
test['tokenized_text'] = test['uncased_text'].apply(tokenizer.tokenize)
train['tokenized_selected_text'] = train['answer_text'].apply(tokenizer.tokenize)